In [1]:
import pyspark


In [2]:
from pyspark.sql import SparkSession as ss
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [3]:
spark = ss.builder.appName('hello').getOrCreate();

In [4]:
spark.readStream.format("kafka")

In [5]:
df = spark.read.csv('../../../Downloads/1500KSalesRecords.csv', inferSchema=True, header=True)

In [11]:
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [9]:
%%time
df.count()

CPU times: user 1.06 ms, sys: 1.4 ms, total: 2.46 ms
Wall time: 3.39 s


5999994

In [27]:
ta = df.createOrReplaceTempView('ta')
tb = df.createOrReplaceTempView('tb')
sql_query = "SELECT * from ta l JOIN tb r ON l.`Order ID` = r.`Order ID` "
left_join = spark.sql(sql_query)

In [23]:
%%time
left_join.count()

CPU times: user 1.21 ms, sys: 1.68 ms, total: 2.89 ms
Wall time: 6.19 s


30974386

In [30]:
%%time
new_column_name_list = left_join.columns
for i, col in enumerate(new_column_name_list):
    count = new_column_name_list.count(col)
    if count > 1:
        idx = new_column_name_list.index(col)
        new_column_name_list[i] = col + '_'+str(i)
left_join = left_join.toDF(*new_column_name_list)
#.coalesce(1)
left_join.write.mode("overwrite").option("header", "true").csv('Downloads/results.csv')

CPU times: user 18.7 ms, sys: 36.2 ms, total: 54.8 ms
Wall time: 1min 53s


In [26]:
%%time
left_join.count()

CPU times: user 1.31 ms, sys: 2.01 ms, total: 3.33 ms
Wall time: 6.25 s


30974386

In [19]:
res_df = spark.read.csv('Downloads/results.csv', inferSchema=True, header=True)

In [20]:
%%time
res_df.count()

CPU times: user 5.75 ms, sys: 6.32 ms, total: 12.1 ms
Wall time: 46.9 s


30974186

In [11]:
df.repartition(1).write.mode("overwrite").format("com.databricks.spark.csv").option("header", "true").csv("test_out.csv")





In [12]:
df.describe()

DataFrame[summary: string, Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: string, Order ID: string, Ship Date: string, Units Sold: string, Unit Price: string, Unit Cost: string, Total Revenue: string, Total Cost: string, Total Profit: string]

In [127]:
df = df.withColumnRenamed("Order ID","order_id")
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [57]:
ta = df.alias('ta')
tb = df.alias('tb')

left_join = ta.join(tb, ta.order_id == tb.order_id ,how='left') # Could also use 'left_outer'
left_join.show()

+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Region|Country|    Item Type|Sales Channel|Order Priority|Order Date| order_id|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Region|Country|    Item Type|Sales Channel|Order Priority|Order Date| order_id|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Europe| Latvia|Personal Care|       Onl

In [12]:
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [23]:



left_join.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit',
 'Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [24]:
newNames = ['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit',
 'RRegion',
 'RCountry',
 'RItem Type',
 'RSales Channel',
 'ROrder Priority',
 'ROrder Date',
 'Rorder_id',
 'RShip Date',
 'RUnits Sold',
 'RUnit Price',
 'RUnit Cost',
 'RTotal Revenue',
 'RTotal Cost',
 'RTotal Profit']
left_join= left_join.toDF(*newNames)
left_join.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit',
 'RRegion',
 'RCountry',
 'RItem Type',
 'RSales Channel',
 'ROrder Priority',
 'ROrder Date',
 'Rorder_id',
 'RShip Date',
 'RUnits Sold',
 'RUnit Price',
 'RUnit Cost',
 'RTotal Revenue',
 'RTotal Cost',
 'RTotal Profit']

In [25]:
left_join.write.csv('Downloads/results.csv')

In [44]:
left_join.show()

+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Region|Country|    Item Type|Sales Channel|Order Priority|Order Date| order_id|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Region|Country|    Item Type|Sales Channel|Order Priority|Order Date| order_id|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+------+-------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Europe| Latvia|Personal Care|       Onl

In [27]:
df.persist(pyspark.StorageLevel.DISK_ONLY)

DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: string, order_id: int, Ship Date: string, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]

In [39]:
df = df.union(df)
df.count()

383999616

In [40]:
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [51]:
sub_df = df.head(100)

In [61]:
test_head = ['Region',
 'Country',
 'Item_Type',
 'Sales_Channel',
 'Order_Priority',
 'Order_Date',
 'order_id',
 'Ship_Date',
 'Units_Sold',
 'Unit_Price',
 'Unit_Cost',
 'Total_Revenue',
 'Total_Cost',
 'Total_Profit']

df.write.parquet('Downloads/test_data.csv' )

SyntaxError: invalid syntax (<ipython-input-61-712e76ef5cd0>, line 16)

In [46]:
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

AttributeError: 'DataFrame' object has no attribute 'reduce'

In [69]:
newDf.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [76]:
for col in df.columns:
    df = df.withColumnRenamed(col, col.replace(' ','_'))
    print(col, col.replace(' ','_'))

df.columns

Region Region
Country Country
Item Type Item_Type
Sales Channel Sales_Channel
Order Priority Order_Priority
Order Date Order_Date
Order ID Order_ID
Ship Date Ship_Date
Units Sold Units_Sold
Unit Price Unit_Price
Unit Cost Unit_Cost
Total Revenue Total_Revenue
Total Cost Total_Cost
Total Profit Total_Profit


['Region',
 'Country',
 'Item_Type',
 'Sales_Channel',
 'Order_Priority',
 'Order_Date',
 'Order_ID',
 'Ship_Date',
 'Units_Sold',
 'Unit_Price',
 'Unit_Cost',
 'Total_Revenue',
 'Total_Cost',
 'Total_Profit']

In [77]:
df.write.parquet('Downloads/test_data.csv' )

In [82]:
test_1 = sp1.read.parquet('Downloads/test_data.csv' );
test_1.count()

17999982

In [81]:
df.write.mode('append').parquet('Downloads/test_data.csv' )

In [89]:
import threading

class ReadRecords(threading.Thread):
    def __init__(self, spark):
        threading.Thread.__init__(self)
        self.spark = spark

    def run(self):
        test_1 = self.spark.read.parquet('Downloads/test_data.csv' )
        print(test_1.count())


thread1 = myThread(sp1)


In [90]:
thread1.start()

17999982


In [ ]:

import pyspark
import threading
class ReadRecords(threading.Thread):
    def __init__(self, spark, topic, idx, schema, input_dict=None, block_params=None, optional_arg=None):
        threading.Thread.__init__(self)
        self.spark = spark
        self.topic = topic
        self.idx = str(idx)
        self.schema = schema
        self.input_dict = input_dict
        self.block_params = block_params
        self.optional_arg = optional_arg

    def run(self):
        self.logger.info("Started reading records for topic", self.topic)
        channels = self.optional_arg["channels"]
        read_msgs_channel = {
            "channelId": channels[self.idx],
            "registerId": ""
        }
        converted_list = []
        while True:
            print('Reading...')
            read_msgs_res = optional_arg['api_instance'].read_messages_from_topic_using_post(read_msgs_channel)
            msgs = read_msgs_res.result
            if (len(msgs) == 0):
                topic = {"topicName": topic}
                res = optional_arg['api_instance'].get_producer_status_using_post(topic)
                if not res.result['value']:
                    break;
            
            for msg in msgs:
                msg = json.loads(msg)
                for index, i in enumerate(msg):
                    if self.field_list[self.idx][index] == 'int':
                        msg[index] = int(float(msg[index]))
                    elif self.field_list[self.idx][index] == 'float':
                        msg[index] = float(msg[index])
                converted_list.append(msg)
            
            if len(converted_list)>100000:
                print('Writing to file',"/tmp/"+ self.idx ,len(converted_list))
                df = self.spark.createDataFrame(converted_list, self.schema)
                df.write.mode('append').parquet('/tmp/'+self.idx)
    
        if len(converted_list)>0:
            print('Writing to file',"/tmp/"+ self.idx ,len(converted_list))
            df = self.spark.createDataFrame(converted_list, self.schema)
            df.write.mode('append').parquet('/tmp/'+self.idx)
        
        print('Done with ', self.idx)



In [92]:
left_join.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit',
 'Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [104]:
left_cols = df.columns.copy()
right_cols = df.columns.copy()
print(left_cols)
print(right_cols)
renamed_cols = {}
new_column_name_list = left_join.columns
for col in new_column_name_list:
    count = new_column_name_list.count(col)
    if(count>1):
        idx = new_column_name_list.index(col)
        new_column_name_list[idx] = 'bbl_'+col

print(new_column_name_list)

['Region', 'Country', 'Item_Type', 'Sales_Channel', 'Order_Priority', 'Order_Date', 'Order_ID', 'Ship_Date', 'Units_Sold', 'Unit_Price', 'Unit_Cost', 'Total_Revenue', 'Total_Cost', 'Total_Profit']
['Region', 'Country', 'Item_Type', 'Sales_Channel', 'Order_Priority', 'Order_Date', 'Order_ID', 'Ship_Date', 'Units_Sold', 'Unit_Price', 'Unit_Cost', 'Total_Revenue', 'Total_Cost', 'Total_Profit']
['bb_lbbl_Region', 'bb_lbbl_Country', 'bb_lItem Type', 'bb_lSales Channel', 'bb_lOrder Priority', 'bb_lOrder Date', 'bb_lorder_id', 'bb_lShip Date', 'bb_lUnits Sold', 'bb_lUnit Price', 'bb_lUnit Cost', 'bb_lTotal Revenue', 'bb_lTotal Cost', 'bb_lTotal Profit', 'bbl_Region', 'bbl_Country', 'Item Type', 'Sales Channel', 'Order Priority', 'Order Date', 'order_id', 'Ship Date', 'Units Sold', 'Unit Price', 'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit']


In [105]:
m_left_join = left_join.toDF(*new_column_name_list)

In [106]:
m_left_join.columns

['bb_lbbl_Region',
 'bb_lbbl_Country',
 'bb_lItem Type',
 'bb_lSales Channel',
 'bb_lOrder Priority',
 'bb_lOrder Date',
 'bb_lorder_id',
 'bb_lShip Date',
 'bb_lUnits Sold',
 'bb_lUnit Price',
 'bb_lUnit Cost',
 'bb_lTotal Revenue',
 'bb_lTotal Cost',
 'bb_lTotal Profit',
 'bbl_Region',
 'bbl_Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [108]:
m_left_join.foreachPartition( ele =>
   ele.grouped(1000).foreach { chunk =>
   print(chunk))

SyntaxError: invalid syntax (<ipython-input-108-8f489a2393e8>, line 1)

In [110]:
from toolz.itertoolz import partition_all

def doSomething(xs):
    print(xs)

for xs in partition_all(batchSize, m_left_join.toLocalIterator()):
    doSomething(xs)

ModuleNotFoundError: No module named 'toolz'

In [111]:
import pandas as pd

In [122]:

import os
directory = 'Downloads/test_data.csv'
for filename in os.listdir(directory):
    pri
    if filename.endswith(".csv"):
        file_path = directory + '/' + filename
        print(filename)
        pdf = pd.read_csv(directory + filename)
        print(len(pdf))
        continue
    else:
        continue



NameError: name 'pri' is not defined

In [123]:
left_join.show()

+----------+-----------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+----------+-----------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|bbl_Region|bbl_Country|    Item Type|Sales Channel|Order Priority|Order Date| order_id|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|bbl_Region|bbl_Country|    Item Type|Sales Channel|Order Priority|Order Date| order_id|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+----------+-----------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+----------+-----------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------

In [134]:
df.coalesce(1).write.mode("overwrite").option("header","true").csv('Downloads/test_left_data.csv' )

In [129]:
full_spark


remove oracle writer



NameError: name 'full_spark' is not defined

In [131]:
for filename in os.listdir('Downloads/test_left_data.csv'):
    print(filename)
    if filename.endswith('.csv'):
        print(filename)

._SUCCESS.crc
part-00000-28b2774b-7ea3-49fb-a01b-9ee093825a02-c000.csv
part-00000-28b2774b-7ea3-49fb-a01b-9ee093825a02-c000.csv
.part-00000-28b2774b-7ea3-49fb-a01b-9ee093825a02-c000.csv.crc
_SUCCESS


In [135]:
rdd_df = df.rdd.zipWithIndex()

In [140]:
df_final = rdd_df.toDF()
df_final.columns

['_1', '_2']

In [154]:
from pyspark.sql.functions import col

temp = df_final.filter(df_final._2>0 and df_final<100))

SyntaxError: invalid syntax (<ipython-input-154-d3d47ee3919c>, line 3)

In [152]:
%%time
temp.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|[Middle East and ...|  1|
|[Australia and Oc...|  2|
+--------------------+---+

CPU times: user 4.36 ms, sys: 2.65 ms, total: 7.01 ms
Wall time: 50 s


In [155]:
df.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'order_id',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [ ]:
column_list = ['order_id', 'Item Type']

df.coalesce(1).write.partitionBy(column_list).mode("overwrite").option("header","true").csv('Downloads/test_left_data.csv' )
